In [66]:
# Carga de datos
import pandas as pd

df_repuestos = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\Lineas de Mantenimiento - repuestos.xlsx")
df_mant = pd.read_excel(r"C:\Users\danie\OneDrive\IA\Proyecto_Final_IA\data\INFORME DE ESTADO DE COMUNICACIONES NO COMUNICACIONES E INFORMATICA ( Br Bl I).xlsx",header=3)

In [67]:
# Mostrar los datos
df_repuestos.head()

,EFECTO (NNE),TIPO DE FALLA,RESEÑA DE LA FALLA,REPUESTOS MAS PROBABLES,REPUESTOS CASI PROBABLES,REPUESTOS POCO PROBABLES
0,MOTOROLA 5050,rx,El equipo transmite pero no recibe,modulo,transistor pnp,NaN
1,MOTOROLA 5050,rx,El equipo transmite pero no recibe,PTT,NaN,NaN
2,MOTOROLA 5050,rx,El equipo transmite pero no recibe,antena,NaN,NaN
3,MOTOROLA 5050,no enciende,El equipo no enciende,placa de alimentacion,bateria,NaN
4,MOTOROLA 5050,no enciende,El equipo no enciende,pin de carga,NaN,NaN


In [68]:
df_mant.head()

,N° Ord,NNE,INE,BAL TANDIL,Unnamed: 4,Unnamed: 5,ESC COM BL 1,Unnamed: 7,Unnamed: 8,CDO BR BL I,...,JEF AGR AA EJ 601 - EC,Unnamed: 43,Unnamed: 44,G MANT SIS AA 601,Unnamed: 46,Unnamed: 47,TOTAL GENERAL,Unnamed: 49,Unnamed: 50,INDIQUE FALLAS
0,NaN,NaN,NaN,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,...,Cargo,E/S,F/S,Cargo,E/S,F/S,Cargo,E/S,F/S,PRINCIPALES
1,1,3895002526898,EQ DE TEN Y REPL RL 31 P/CABLE TEF CAMP,NaN,NaN,NaN,7,7,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,8,8,0,NaN
2,2,3895003563937,EJE DE CARRETE DE TEN Y REPL RL 27 D P/CABLE T...,NaN,NaN,NaN,7,7,0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7,7,0,NaN
3,3,3895005377953,EQ DE TEN Y REPL RL 26 P/CABLE TEF CAMP,NaN,NaN,NaN,1,0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,1,UM (CUENTA MUSEO ) (ESC COM BL 1)
4,4,5805005211320,TELEFONO DE CAMPAÑA PORTATIL TA 1/PT,NaN,NaN,NaN,11,0,11,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,11,0,11,UM (CUENTA MUSEO ) (ESC COM BL 1)
